In [ ]:
import os

import numpy as np
import pandas as pd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
os.chdir('/home/adam/adam/causal_inference')

from causal_inference.make_data.make_raw_data import UseCaseLoader
from causal_inference.make_data.make_raw_data import COLUMNS_POSITION

from causal_inference.make_data.make_proning_sessions import *

In [ ]:
dl = UseCaseLoader()

In [ ]:
path = '/home/adam/adam/data/causal_inference/data/raw/'
buf = 'position_measurements.csv'

In [ ]:
dl.get_position_measurements(path=path, path_or_buf=buf)

In [ ]:
dtype = {'hash_patient_id': object, 'episode_id': object, 'pacmed_subname': object, 'effective_value': object,
             'numerical_value': np.float64, 'is_correct_unit_yn': bool, 'unit_name': object, 'hospital': object,
             'ehr':object}

In [ ]:
df = pd.read_csv(path+buf, index_col=False, dtype=dtype, parse_dates=['start_timestamp', 'end_timestamp'])


In [ ]:
df_batch = df.loc[(df['episode_id'] == df.episode_id.sample(1).iloc[0]) &
                  (df['pacmed_subname'] == 'position_body')]
df_batch = df_batch.sort_values(by=['start_timestamp'], ascending=True).reset_index(drop=True)
df_batch['new_session'] = False
df_batch['effective_value_previous'] = False
df_batch.loc[:, 'effective_value_previous'] = df_batch.loc[:, 'effective_value'].shift(1)
df_batch.loc[1:, 'new_session'] = df_batch.loc[1:, 'effective_value'] != df_batch.loc[1:, 'effective_value_previous']
df_batch['new_session'] = df_batch['new_session'].astype(int).cumsum()
df_batch['hash_session_id'] = df_batch['hash_patient_id'] + df_batch['new_session'].astype(str)
df_batch[['effective_value', 'new_session', 'start_timestamp']].head(60)

In [ ]:
df_batch.head()

In [ ]:
df_batch.groupby('hash_session_id').agg(start_timestamp=pd.NamedAgg(column="start_timestamp", aggfunc="min"),
                                        end_timestamp=pd.NamedAgg(column="start_timestamp", aggfunc="max")).reset_index()


In [ ]:
df_batch['end_timestamp_extracted'] = df_batch['start_timestamp'].shift(-1)
df_batch.loc[df_batch.index[-1], 'end_timestamp_extracted'] = df_batch.loc[df_batch.index[-1], 'start_timestamp']
df_batch.loc[df_batch.end_timestamp.isnull(), 'end_timestamp'] = df_batch.loc[df_batch.end_timestamp.isna(), 'end_timestamp_extracted']

In [ ]:
df_batch.tail(5)

In [ ]:
df_batch.info()
